In [1]:
from config import *
from model import *
from data_preprocessing import *
from correction import *

torch.manual_seed(0)

In [3]:
# load data
train_data, test_data, anomal_idx = data_preprocessing(args.train_data, args.test_data)

train_data, validate_data = Split_data(train_data)

#데이터 셋을 구성함.
train_dataset = Get_Dataset(train_data, args.window_size)
validate_dataset = Get_Dataset(validate_data, args.window_size)
test_dataset = Get_Dataset(test_data, args.window_size)

#로더 형태로 변환함
train_loader = torch.utils.data.DataLoader(
                 dataset=train_dataset,
                 batch_size=args.batch_size,
                 shuffle=True)
valid_loader = torch.utils.data.DataLoader(
                dataset=validate_dataset,
                batch_size=args.batch_size,
                shuffle=False)
test_loader = torch.utils.data.DataLoader(
                dataset=test_dataset,
                batch_size=args.batch_size,
                shuffle=False)

# VAE-GAN

In [7]:
model_VAE, model_discriminator, model_generator = Model_initialize(input_dim=args.input_size, latent_dim2=args.latent_size2, latent_dim=args.latent_size, window_size=args.window_size, num_layers=args.num_layers, batch_size=args.batch_size)
model_VAE.to(args.device)
model_discriminator.to(args.device)
model_generator.to(args.device)

## 학습하기
if args.Mode == 'Train':
    model_VAE, model_discriminator, model_generator = run(args, model_VAE, model_discriminator, model_generator, train_loader, valid_loader)

Generator(
  (lstm): LSTM(14, 100, num_layers=2, batch_first=True)
  (relu): ReLU()
  (fc): Linear(in_features=100, out_features=14, bias=True)
)

In [9]:
args.Mode == 'Test'

if args.Mode == 'Test':
    model_VAE = torch.load(VAE)
    model_discriminator = torch.load(Discriminator_model)
    model_generator = torch.load(Generator_model)
    model_VAE.to(args.device)
    model_discriminator.to(args.device)
    model_generator.to(args.device)
    
Score,predictions,true_data = Get_Threshold(args, model_VAE, model_generator, model_discriminator, test_loader, 0, 0.5)
THRESHOLD = 0.5
correct_anomaly = get_Accuracy(Score, anomal_idx, THRESHOLD,0)

# Data prediction

In [15]:
x_train, x_val, y_train, y_val = build_dataset(np.array(train_data), args.seq_length, args.n_steps_out)

prediction_model = proposed_prediction(x_train, args.seq_length, args.n_steps_out)
prediction_model.compile(loss='mse', optimizer='adam')

history = prediction_model.fit(x_train, y_train, epochs=100, batch_size=64, validation_data=(x_val, y_val), verbose=0, shuffle=False, 
                    callbacks=[ModelCheckpoint(monitor='val_loss', filepath='models/swat_AttentionLSTM_14.hdf5', verbose=0, save_best_only=True), 
                              EarlyStopping(monitor='val_loss', patience=5, verbose=0, mode='auto')])

In [19]:
start_idx = args.seq_length 
_, test_data, anomal_idx = data_preprocessing_test(args.train_data, args.test_data, start_idx, args.seq_length)
test_X, test_Y = build_dataset_uni(test_data, args.seq_length)

y_hat = prediction_model.predict(test_X)
y_trans = y_hat.reshape(-1,test_X.shape[2])
y_real = test_Y

test_data_front = test_data[:args.seq_length,:]
test_data_back = test_data[len(test_data_front) + len(y_trans) + int(args.n_steps_out):start_idx,:]

y_hat_final = np.concatenate([np.array(test_data_front), y_trans], axis=0)
y_hat_final = np.concatenate([y_hat_final, np.array(test_data_back)], axis=0)

# Data Correction

In [24]:
y_hat_data = Get_Dataset(y_hat_final, args.window_size)

predict_loader = torch.utils.data.DataLoader(
                dataset=y_hat_data,
                # batch_size=args.batch_size,
                batch_size=1,
                shuffle=False)

test_loader = torch.utils.data.DataLoader(
                dataset=test_dataset,
                # batch_size=args.batch_size,
                batch_size=1,
                shuffle=False)

predict_iterator = enumerate(predict_loader) # test_loader

predict_value = []
for i, batch_data in predict_iterator:
    # print(i)
    predict_value.append(batch_data.to(args.device))
    # real_value.append(batch_data.to(args.device))
    
predicted_data = predictions.copy()
real_data = true_data.copy()

predictions_v1,true_data_v1 = Data_Correction(Score, correct_anomaly, THRESHOLD, args, model_VAE, model_generator, predicted_data, real_data, predict_value, test_loader, 0)

In [34]:
# save data to pickle
dir_fold = './data'
if not os.path.exists(dir_fold):
    os.mkdir(dir_fold)
    
with gzip.open( dir_fold + "/corrected_data.pickle", "wb" ) as file:
    pickle.dump(predictions_v1.cpu().detach().numpy(), file)